In [3]:
import hashlib
import requests
import time
from tqdm import tqdm
import os
import socket

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def requests_retry_session(retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


def get_pin_hash(pin):
    pin_str = f"{pin:06d}"  # 6-digit format
    return hashlib.sha1(pin_str.encode('utf-8')).hexdigest().upper()

def check_pin(pin):
    pin_hash = get_pin_hash(pin)
    prefix = pin_hash[:5]
    suffix = pin_hash[5:]
    
    url = f"https://api.pwnedpasswords.com/range/{prefix}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Request error encountered: {e}")
        time.sleep(10)  # Wait for 10 seconds before retrying
        return None
    except socket.gaierror as e:
        print(f"DNS resolution error: {e}")
        time.sleep(10)  # Wait for 10 seconds before retrying
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        time.sleep(10)  # Wait for 10 seconds before retrying
        return None
    
    hashes = (line.split(':') for line in response.text.splitlines())
    for h, count in hashes:
        if h == suffix:
            return int(count)
    return 0

def save_checkpoint(pin, results):
    with open('checkpoint.txt', 'w') as f:
        f.write(f"{pin}\n")
        for pin, count in results.items():
            f.write(f"{pin}:{count}\n")

def load_checkpoint():
    if os.path.exists('checkpoint.txt'):
        with open('checkpoint.txt', 'r') as f:
            lines = f.readlines()
            last_pin = int(lines[0].strip())
            results = {}
            for line in lines[1:]:
                pin, count = line.strip().split(':')
                results[pin] = int(count)
            return last_pin, results
    return 0, {}

# Load the last checkpoint
start_pin, results = load_checkpoint()

# Main loop
for pin in tqdm(range(start_pin, 1000000)):  # 6-digit PINs
    count = check_pin(pin)
    if count is not None:  # Proceed if the request was successful
        if count > 0:
            results[f"{pin:06d}"] = count  # Save the result
        if pin % 100 == 0:  # Save checkpoint every 100 PINs
            save_checkpoint(pin, results)
#    time.sleep(0.1)  # Sleep for [0.2] 200ms between requests

# Final save of results
save_checkpoint(1000000, results)

# Sort results by PIN (smallest to largest)
sorted_by_pin = sorted(results.items(), key=lambda x: x[0])

# Sort results by count (largest to smallest)
sorted_by_count = sorted(results.items(), key=lambda x: x[1], reverse=True)

# Write all PINs with counts to a text file
with open('all_6digit_pins_with_counts.txt', 'w') as f:
    for pin, count in sorted_by_pin:
        f.write(f"{pin} : {count}\n")

# Write sorted PINs (by count) to a separate text file
with open('6digit_pins_sorted_by_count.txt', 'w') as f:
    for pin, _ in sorted_by_count:
        f.write(f"{pin}\n")

print("Scan complete. Results have been written to 'all_6digit_pins_with_counts.txt' and '6digit_pins_sorted_by_count.txt'.")


  0%|          | 232/702800 [01:37<1609:29:21,  8.25s/it]

Request error encountered: HTTPSConnectionPool(host='api.pwnedpasswords.com', port=443): Max retries exceeded with url: /range/EDDEE (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x751a2df18f50>: Failed to resolve 'api.pwnedpasswords.com' ([Errno -3] Temporary failure in name resolution)"))


  0%|          | 272/702800 [02:34<877:32:01,  4.50s/it] 

Request error encountered: HTTPSConnectionPool(host='api.pwnedpasswords.com', port=443): Max retries exceeded with url: /range/4C635 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x751a2df58250>: Failed to resolve 'api.pwnedpasswords.com' ([Errno -3] Temporary failure in name resolution)"))


  0%|          | 273/702800 [02:54<1785:44:51,  9.15s/it]

Request error encountered: HTTPSConnectionPool(host='api.pwnedpasswords.com', port=443): Max retries exceeded with url: /range/04695 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x751a2df5bb10>: Failed to resolve 'api.pwnedpasswords.com' ([Errno -3] Temporary failure in name resolution)"))


  0%|          | 925/702800 [06:20<391:28:04,  2.01s/it] 

Request error encountered: HTTPSConnectionPool(host='api.pwnedpasswords.com', port=443): Max retries exceeded with url: /range/DF96C (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x751a2df1b2d0>: Failed to resolve 'api.pwnedpasswords.com' ([Errno -3] Temporary failure in name resolution)"))


  0%|          | 925/702800 [06:39<84:08:08,  2.32it/s] 


KeyboardInterrupt: 

In [1]:
import hashlib
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm
import os
import time

def create_session():
    session = requests.Session()
    retry = Retry(total=3, backoff_factor=0.3, status_forcelist=[500, 502, 504])
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_pin_hash(pin):
    pin_str = f"{pin:06d}"
    return hashlib.sha1(pin_str.encode('utf-8')).hexdigest().upper()

def check_pins(session, pins):
    results = {}
    for pin in pins:
        pin_hash = get_pin_hash(pin)
        prefix = pin_hash[:5]
        suffix = pin_hash[5:]
        
        url = f"https://api.pwnedpasswords.com/range/{prefix}"
        
        try:
            response = session.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request error encountered: {e}")
            time.sleep(10)
            continue
        
        hashes = (line.split(':') for line in response.text.splitlines())
        for h, count in hashes:
            if h == suffix:
                results[f"{pin:06d}"] = int(count)
                break
    
    return results

def save_checkpoint(pin, results):
    with open('checkpoint.txt', 'w') as f:
        f.write(f"{pin}\n")
        for pin, count in results.items():
            f.write(f"{pin}:{count}\n")

def load_checkpoint():
    if os.path.exists('checkpoint.txt'):
        with open('checkpoint.txt', 'r') as f:
            lines = f.readlines()
            last_pin = int(lines[0].strip())
            results = {}
            for line in lines[1:]:
                pin, count = line.strip().split(':')
                results[pin] = int(count)
            return last_pin, results
    return 0, {}

def main():
    start_pin, results = load_checkpoint()
    session = create_session()

    for batch_start in tqdm(range(start_pin, 1000000, 100)):
        batch_end = min(batch_start + 100, 1000000)
        batch_results = check_pins(session, range(batch_start, batch_end))
        results.update(batch_results)
        save_checkpoint(batch_end, results)
        time.sleep(1)  # Sleep for 1 second between batches

    # Sort and save results
    sorted_by_pin = sorted(results.items(), key=lambda x: x[0])
    sorted_by_count = sorted(results.items(), key=lambda x: x[1], reverse=True)

    with open('all_6digit_pins_with_counts.txt', 'w') as f:
        for pin, count in sorted_by_pin:
            f.write(f"{pin} : {count}\n")

    with open('6digit_pins_sorted_by_count.txt', 'w') as f:
        for pin, _ in sorted_by_count:
            f.write(f"{pin}\n")

    print("Scan complete. Results have been written to 'all_6digit_pins_with_counts.txt' and '6digit_pins_sorted_by_count.txt'.")

if __name__ == "__main__":
    main()

0it [00:00, ?it/s]


Scan complete. Results have been written to 'all_6digit_pins_with_counts.txt' and '6digit_pins_sorted_by_count.txt'.
